# Test Case for Implementing a Diffusion Operator Coupling Field Components
### Objective
We aim to implement a VDF discretization of a diffusion operator coupling the components of a field:
$$
\nabla\cdot(D(\mathbf{x})\nabla\phi(\mathbf{x})),
$$
where the component $a$ is given by:
$$
\sum_{i=1}^d\sum_{b=1}^{N}\partial_i(D_{ab}(\mathbf{x})\partial_i\phi_b(\mathbf{x})),
$$
where $N$ is the number of components and $d$ is the dimension of the space.

An appropriate test would be solving the equation:
$$
-\nabla\cdot(D\nabla\phi)=S
$$
in a simple case which has an analytical solution.

### Test for uniform diffusion coefficients
We could consider the following cases where:
- the domain of resolution is $\Omega=[-1,1]^d$, and we impose the boundary condition:
  $$
  \phi=0 \quad \text{on} \quad \partial\Omega;
  $$
- $D$ is uniform and invertible;
- $S=DS'$ with
  $$
  S'_a = \prod_{i=1}^d\sin(\pi k_ax_i),
  $$
  where the $k_a \in \mathbb{N}^*$.

In this case, by multiplying the equation $\nabla\cdot(D\nabla\phi)=S$ by $D^{-1}$, we find that it is equivalent to a system of decoupled Poisson equations:
$$
\Delta\phi_a=-S'_a.
$$
It is known that with the boundary condition there is a unique solution for each Poisson equation, and thus a unique solution of the system. The form of the sources was chosen so that the solution is evident:
$$
\phi_a = \frac{S'_a}{d\pi^2k_a^2}.
$$

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse, Ayoub Ouazzani")
run.TRUST_parameters()
run.reset()

### Test case

Takes about 10s for default parameters.

In [ ]:
import numpy as np

##### PARAMETERS #####
# space_dim = 2
nb_comp = 3
nb_nodes = 100
seuil_statio = 1e-3

# diffusion matrix
diagD = 1.0
nondiagD = 0.1
D = diagD * np.identity(nb_comp) + nondiagD * (np.ones((nb_comp, nb_comp)) - np.identity(nb_comp))

# time scheme
implicit = True
#####################

Dl = list(np.resize(D, (np.size(D),)))

sch = "sch_e"
if implicit:
    sch = "sch_i"

kv = [1 + i for i in range(nb_comp)]
x = ("x", "y")

source = str(nb_comp) + " "
sol = str(nb_comp) + " "
Sp = []
for a in range(nb_comp):
    Sap = "*".join([f"sin(pi*{kv[a]}*{xj})" for xj in x])
    Sp.append(Sap)
    sol += Sap + f"/2/pi/pi/{kv[a]}/{kv[a]} "
for a in range(nb_comp):
    source += "+".join([f"{D[a,b]}*" + Sp[b] for b in range(nb_comp)]) + " "

himp = np.identity(nb_comp)
himp *= 1e10
flattened_list = himp.flatten().tolist()

subs = {
    "nb_nodes": nb_nodes,
    "sch": sch,
    "seuil_statio": seuil_statio,
    "Dcoeff": " ".join(map(str, [len(Dl)] + Dl)),
    "cond_init": " ".join(map(str, [nb_comp] + [0] * nb_comp)),
    "cond_lim": " ".join(map(str, [nb_comp] + [0] * nb_comp)),
    "h_imp": " ".join(map(str, [len(Dl)] + flattened_list)),
    "source": source,
    "sol": sol,
}

run.addCaseFromTemplate("jdd.data", "results", subs)

run.runCases()

### Plot of comparison to exact solution 



In [ ]:
from trustutils import plot
import matplotlib.colors as mcolors
# c = list(mcolors.TABLEAU_COLORS.keys())

for i in range(nb_comp):
    a = plot.Graph()
    a.addSegment(f"{run.BUILD_DIRECTORY}/results/jdd_CONCENTRATION.son", label="Numerical", compo=i)
    a.addSegment(f"{run.BUILD_DIRECTORY}/results/jdd_CONCENTRATION_EX.son", label="Exact", compo=i)